<a href="https://colab.research.google.com/github/udlbook/udlbook/blob/main/Notebooks/Chap12/12_2_Multihead_Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.2: Multhead Self-Attention**

This notebook builds a multihead self-attention mechanism as in figure 12.6

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.



In [1]:
import numpy as np
import matplotlib.pyplot as plt

The multihead self-attention mechanism maps $N$ inputs $\mathbf{x}_{n}\in\mathbb{R}^{D}$ and returns $N$ outputs $\mathbf{x}'_{n}\in \mathbb{R}^{D}$.  



In [2]:
# Set seed so we get the same random numbers
np.random.seed(3)
# Number of inputs
N = 6
# Number of dimensions of each input
D = 8
# Create an empty list
X = np.random.normal(size=(D,N))
# Print X
print(X)

[[ 1.78862847  0.43650985  0.09649747 -1.8634927  -0.2773882  -0.35475898]
 [-0.08274148 -0.62700068 -0.04381817 -0.47721803 -1.31386475  0.88462238]
 [ 0.88131804  1.70957306  0.05003364 -0.40467741 -0.54535995 -1.54647732]
 [ 0.98236743 -1.10106763 -1.18504653 -0.2056499   1.48614836  0.23671627]
 [-1.02378514 -0.7129932   0.62524497 -0.16051336 -0.76883635 -0.23003072]
 [ 0.74505627  1.97611078 -1.24412333 -0.62641691 -0.80376609 -2.41908317]
 [-0.92379202 -1.02387576  1.12397796 -0.13191423 -1.62328545  0.64667545]
 [-0.35627076 -1.74314104 -0.59664964 -0.58859438 -0.8738823   0.02971382]]


We'll use two heads.  We'll need the weights and biases for the keys, queries, and values (equations 12.2 and 12.4).  We'll use two heads, and (as in the figure), we'll make the queries keys and values of size D/H

In [3]:
# Number of heads
H = 2
# QDV dimension
H_D = int(D/H)

# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters for the first head
omega_q1 = np.random.normal(size=(H_D,D))
omega_k1 = np.random.normal(size=(H_D,D))
omega_v1 = np.random.normal(size=(H_D,D))
beta_q1 = np.random.normal(size=(H_D,1))
beta_k1 = np.random.normal(size=(H_D,1))
beta_v1 = np.random.normal(size=(H_D,1))

# Choose random values for the parameters for the second head
omega_q2 = np.random.normal(size=(H_D,D))
omega_k2 = np.random.normal(size=(H_D,D))
omega_v2 = np.random.normal(size=(H_D,D))
beta_q2 = np.random.normal(size=(H_D,1))
beta_k2 = np.random.normal(size=(H_D,1))
beta_v2 = np.random.normal(size=(H_D,1))

# Choose random values for the parameters
omega_c = np.random.normal(size=(D,D))

Now let's compute the multiscale self-attention

In [4]:
# Define softmax operation that works independently on each column
def softmax_cols(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in)
  # Sum over columns
  denom = np.sum(exp_values, axis = 0)
  # Compute softmax (numpy broadcasts denominator to all rows automatically)
  softmax = exp_values / denom
  # return the answer
  return softmax

In [5]:
 # Now let's compute self attention in matrix form
def multihead_scaled_self_attention(X,omega_v1, omega_q1, omega_k1, beta_v1, beta_q1, beta_k1, omega_v2, omega_q2, omega_k2, beta_v2, beta_q2, beta_k2, omega_c):

  # TODO Write the multihead scaled self-attention mechanism.
  # Replace this line
  V_1 = beta_v1 + omega_v1 @ X
  V_2 = beta_v2 + omega_v2 @ X
  Q_1 = beta_q1 + omega_q1 @ X
  Q_2 = beta_q2 + omega_q2 @ X
  K_1 = beta_k1 + omega_k1 @ X
  K_2 = beta_k2 + omega_k2 @ X
  attention_1 = softmax_cols(K_1.T @ Q_1 / np.sqrt(Q_1.shape[0]))
  attention_2 = softmax_cols(K_2.T @ Q_2 / np.sqrt(Q_2.shape[0]))
  X_prime1 = V_1 @ attention_1
  X_prime2 = V_2 @ attention_2
  X_prime = omega_c @ np.concatenate((X_prime1, X_prime2), axis=0)

  return X_prime

In [6]:
# Run the self attention mechanism
X_prime = multihead_scaled_self_attention(X,omega_v1, omega_q1, omega_k1, beta_v1, beta_q1, beta_k1, omega_v2, omega_q2, omega_k2, beta_v2, beta_q2, beta_k2, omega_c)

# Print out the results
np.set_printoptions(precision=3)
print("Your answer:")
print(X_prime)

print("True values:")
print("[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]")
print(" [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]")
print(" [  5.479   1.115   9.244   0.453   5.656   7.089]")
print(" [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]")
print(" [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]")
print(" [  3.548  10.036  -2.244   1.604  12.113  -2.557]")
print(" [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]")
print(" [  1.248  18.894  -6.409   3.224  19.717  -5.629]]")

# If your answers don't match, then make sure that you are doing the scaling, and make sure the scaling value is correct

Your answer:
[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]
 [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]
 [  5.479   1.115   9.244   0.453   5.656   7.089]
 [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]
 [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]
 [  3.548  10.036  -2.244   1.604  12.113  -2.557]
 [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]
 [  1.248  18.894  -6.409   3.224  19.717  -5.629]]
True values:
[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]
 [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]
 [  5.479   1.115   9.244   0.453   5.656   7.089]
 [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]
 [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]
 [  3.548  10.036  -2.244   1.604  12.113  -2.557]
 [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]
 [  1.248  18.894  -6.409   3.224  19.717  -5.629]]


## Tensor approach (vectorizing the entire attention) - using 3D matrices
<span style="color:green;white-space:pre-wrap">Honestly feels a bit iffy to do the multihead attention this way as proposed by the author. I've seen other loop over the heads instead and then concatenate the $V_h * softmax_h(K^TQ)$ before multiplying by $\omega_h * (V_h * softmax_h(K^TQ))$</span>

```py
V_1 = beta_v1 + omega_v1 @ X
V_2 = beta_v2 + omega_v2 @ X
Q_1 = beta_q1 + omega_q1 @ X
Q_2 = beta_q2 + omega_q2 @ X
K_1 = beta_k1 + omega_k1 @ X
K_2 = beta_k2 + omega_k2 @ X
attention_1 = softmax_cols(K_1.T @ Q_1 / np.sqrt(Q_1.shape[0]))
attention_2 = softmax_cols(K_2.T @ Q_2 / np.sqrt(Q_2.shape[0]))
X_prime1 = V_1 @ attention_1
X_prime2 = V_2 @ attention_2
X_prime = omega_c @ np.concatenate((X_prime1, X_prime2), axis=0)
```

<span style="color:green;white-space:pre-wrap">While I admit I couldn't make it fully vectorized, mostly because of how the matrices were
constructed originally by the author I still needed to do some extra work on stacking them and also modify the transpose so that it mimics the effect in a 2D matrix but for a 3D tensor. To do my approach you really need to understand what the cols and rols means in the matrix and the 3D tensor and how to manipulate them in a way that makes sense adhering to the attention formulas.</span>

```py
# Stack the matrices first
omega_q = np.stack((omega_q1, omega_q2), axis=0)
omega_k = np.stack((omega_k1, omega_k2), axis=0)
omega_v = np.stack((omega_v1, omega_v2), axis=0)
beta_q = np.stack((beta_q1, beta_q2), axis=0)
beta_k = np.stack((beta_k1, beta_k2), axis=0)
beta_v = np.stack((beta_v1, beta_v2), axis=0)

# now do multi-head self-attention
V = beta_v + omega_v @ X
Q = beta_q + omega_q @ X
K = beta_k + omega_k @ X
# transposed_K = np.stack((K[0, :, :].T, K[1, :, :].T), axis=0)
transposed_K = np.transpose(K, axes=(0, 2, 1))  # this is the same as above, the above actually shows what is happening, we do transpose on all the 2D matrices that are stacked in the 3D tensor. They are stacked along the rows
attention = softmax_cols_tensor(transposed_K @ Q / np.sqrt(H_D))
X_prime = V @ attention
X_prime = X_prime.reshape(-1, X_prime.shape[-1])
X_prime = omega_c @ X_prime
```
<span style="color:green;white-space:pre-wrap">In this particular case I don't think einsum would help that much, it can't be made much shorter, or can it?</span>

<span style="color:green;white-space:pre-wrap">So this way of doing it is shorter, a little more elegant. It's still shorter though, but maybe less illuminating. Thinking more about it and taking a quick glance at other people's code, it seems that you just need a loop over the heads. In my case the bottleneck I think is on the stacking of the matrices, requiring loop over the heads, so they should maybe be about equally fast, but my approach is a bit unclearer because tensors...Although I did learn alot about working with 3D tensors and how numpy works with the rows, cols and depths.</span>

<span style="color:green;white-space:pre-wrap">It would be too tedious to visualize all the 3D tensor manipulations. I will give the most important idea that is sufficient to understand how to think about these operations, and it should be easy to abstract this to the other operations in the self-attention mechanism. I will also show how to think about the transposed operation, which should be self-evident after understanding the first idea I present.</span>

<span style="color:green;white-space:pre-wrap">First idea, finding the 2D matrix in a 3D tensor and then doing matrix multiplication is the same as any 2D matrix with another 2D matrix. First picture is just a 3D tensor 3x3x3 multiplying itself, which can be interpreted as each of the 2D 3x3 matrices inside it in isolation do matrix multiplication with themselves. The second image is square 4x4 with a 3x4x2 tensor with 4x2 rectangular matrix inside the tensor, but the principle is exactly the same as normal matrix multiplication, as soon as you know how to interpret the 2D matrix you will know how to do the operation. Secondly, it should now be quite apparent why switching the last two indices with each other (row and col of the 2D matrix inside the 3D tensor) is the same as transposing if you look at the first picture for instance. Note that numpy interprets the row, col and depth differently than what you mathematically would interpret row and cols. Nevertheless, it doesn't matter, as soon as you know how to interpret the 2D matrix inside the 3D tensor you can treat multiplications with 3D tensors the same way as with 2D matrices to the same degree as shown in the images. In this case the heads will be stacked matrices along the rows, and we can be confident that when you perform matrix multiplication between 3D tensors or 2D matrices with 3D tensors, the matrices will be multiplied as we would expect according to the attention formulas. Finally, the np.sum() over the columns for softmax is also illustrated, how the values are calculated and what form the resulting tensor will have if we set keepdims=True. If not it will turn into a 2D matrix, which can't be broadcast properly. With this we can broadcast it properly over the respective matrices, which is kinda neat.</span>
![3D tensor square matrix](../../public/3D_tenor_square_matrix_mult.png)
![3D tensor rectangular matrix](../../public/3D_tensor_rectangular_matrix_mult.png)
![3D tensor "col" transpose](../../public/transposing_3D_tensor.png)
![3D tensor col summation](../../public/3D_tensor_col_sum.png)

<span style="color:green;white-space:pre-wrap">An easy way to think about using a 3D tensor in this case is we can imagine that we put the heads in a list and instead of having to loop over each head and do the matrix multiplications according to the attention formulas as we did in 12.1, now we can just do one matrix multiplication directly with the 3D tensor and numpy will make sure that the multiplications with the tensors behave nicely, and the key to understand this is to realize that the 2D matrices are stacked row-wise in the tensor and treated in isolation. Because of this, multiplication will be carried out in isolation matrix-wise like they would in 2D * 2D matrix situation, and since we know how to do matrix multiplication with 2D matrices, then we should understand now how to do it for 3D tensors. If this is still not clear, then best way to understand this is just trial and error in a console with numpy, try different 3D tensors and 2D matrices, multiply them together, tranpose them, sum them and make sure you understand how the result is calculated, that's how I did it.</span>

In [7]:
# Number of heads
H = 2
# QDV dimension
H_D = int(D/H)

# Set seed so we get the same random numbers
np.random.seed(0)

# Choose random values for the parameters for both the heads in one 3D tensor
# We have to follow the same seed to get the exact same values, but also the same procedure to create the matrices, so we need to stack the matrices instead of creating tensors. 
# If we create tensors there's no guarantee that we will get the same values, because the author didn't create the data that way
# note that the axis is by default set to 0, but I'm setting it explicitly to make it clearer that we stack along the rows, this will give a 3D tensor
omega_q = np.stack((omega_q1, omega_q2), axis=0)
omega_k = np.stack((omega_k1, omega_k2), axis=0)
omega_v = np.stack((omega_v1, omega_v2), axis=0)
beta_q = np.stack((beta_q1, beta_q2), axis=0)
beta_k = np.stack((beta_k1, beta_k2), axis=0)
beta_v = np.stack((beta_v1, beta_v2), axis=0)

# Choose random values for the parameters
# omega_c doesn't need to be changed, because we should flatten the array before multiplying omega_c with the v * softmax(k.T,Q)
 
 # Define softmax operation that works independently on each column
def softmax_cols_tensor(data_in):
  # Exponentiate all of the values
  exp_values = np.exp(data_in)
  # Sum over rows, because of how the tensor is built and how numpy works with np.sum(), but keep dimension, so broadcast can be done
  denom = np.sum(exp_values, axis = 1, keepdims=True)
  # Compute softmax (numpy broadcasts denominator to all rows automatically)
  softmax = exp_values / denom
  # return the answer
  return softmax
 
 # Now let's compute self attention in tensor form
def multihead_scaled_self_attention_tensor(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k, omega_c):

  # TODO Write the multihead scaled self-attention mechanism.
  # Replace this line
  V = beta_v + omega_v @ X
  Q = beta_q + omega_q @ X
  K = beta_k + omega_k @ X
  #transposed_K = np.stack((K[0, :, :].T, K[1, :, :].T), axis=0)  # for each matrix on the row, transpose it 
  transposed_K = np.transpose(K, axes=(0, 2, 1))  # this is the same as above, the above actually shows what is happening, we do transpose on all the 2D matrices that are stacked in the 3D tensor. They are stacked along the rows
  attention = softmax_cols_tensor(transposed_K @ Q / np.sqrt(H_D))
  X_prime = V @ attention
  X_prime = X_prime.reshape(-1, X_prime.shape[-1])  # flatten the matrix from Hx(D/H)xD to DxD, so that they are concatenated row-wise in a 2D matrix to make a DxD matrix
  X_prime = omega_c @ X_prime

  return X_prime

# Run the self attention mechanism
X_prime = multihead_scaled_self_attention_tensor(X,omega_v, omega_q, omega_k, beta_v, beta_q, beta_k, omega_c)

# Print out the results
np.set_printoptions(precision=3)
print("Your answer:")
print(X_prime)

print("True values:")
print("[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]")
print(" [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]")
print(" [  5.479   1.115   9.244   0.453   5.656   7.089]")
print(" [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]")
print(" [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]")
print(" [  3.548  10.036  -2.244   1.604  12.113  -2.557]")
print(" [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]")
print(" [  1.248  18.894  -6.409   3.224  19.717  -5.629]]")

Your answer:
[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]
 [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]
 [  5.479   1.115   9.244   0.453   5.656   7.089]
 [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]
 [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]
 [  3.548  10.036  -2.244   1.604  12.113  -2.557]
 [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]
 [  1.248  18.894  -6.409   3.224  19.717  -5.629]]
True values:
[[-21.207  -5.373 -20.933  -9.179 -11.319 -17.812]
 [ -1.995   7.906 -10.516   3.452   9.863  -7.24 ]
 [  5.479   1.115   9.244   0.453   5.656   7.089]
 [ -7.413  -7.416   0.363  -5.573  -6.736  -0.848]
 [-11.261  -9.937  -4.848  -8.915 -13.378  -5.761]
 [  3.548  10.036  -2.244   1.604  12.113  -2.557]
 [  4.888  -5.814   2.407   3.228  -4.232   3.71 ]
 [  1.248  18.894  -6.409   3.224  19.717  -5.629]]


<span style="color:green;white-space:pre-wrap">The benefit of using a loop on the multi-head instead is that you don't need to know anything about how matrix multiplication with 3D tensors works, so that approach is clearer. With this approach you still need a loop I think for stacking up the matrices V,Q,K? I think that or you can just initialize as 3D tensors and maybe that would work?</span>